before we start, we first specify our attributes: (in attributes.py file)

In [ ]:
attributes = [
    ['low', 'med', 'high', 'vhigh'],  # buying
    ['low', 'med', 'high', 'vhigh'],  # maint
    ['2', '3', '4', '5more'],  # doors
    ['2', '4', 'more'],  # persons
    ['small', 'med', 'big'],  # lug_boot
    ['low', 'med', 'high'],  # safety

    ['unacc', 'acc', 'good', 'vgood'],  # label
]

then we create an Object class to map raw data into an Object (in object.py file)

In [ ]:
from attributes import attributes

class Object:
    def __init__(self, splits) -> None:
        self.attributes = []
        for index in range(len(attributes)):
            value = attributes[index].index(splits[index])
            self.attributes.append(value)

the main logic of application will be inside the Node class (in node.py file)

in node.py we first import our required codes:

In [ ]:
from math import log

from objects import Object
from attributes import attributes

here we introduce `Node` class:

In [ ]:
class Node:
    def __init__(self,  objects: list[Object], max_depth: int) -> None:
        self.objects = objects
        self.max_depth = max_depth
        self.label = -1

in the `split` function we recursievly split the passed objects.

In [ ]:
    def split(self, available_attributes: list[int]) -> None:
        maj = self.majority(self.objects)
        we_reach = True

        for obj in self.objects:
            if (obj.attributes[-1] != maj):
                we_reach = False
                break

        depth = (len(attributes) - 1) - len(available_attributes)
        if (we_reach or self.max_depth == depth):
            self.label = maj
            return

        splits = []  # contain all splits of nodes

        for attribute in available_attributes:
            nodes = []  # contain nodes with this attribute

            for attribute_value in range(len(attributes[attribute])):
                matches = filter(
                    lambda obj: obj.attributes[attribute] == attribute_value, self.objects)
                nodes.append(Node(list(matches), self.max_depth))

            splits.append(nodes)

        available_attributes_for_children = available_attributes.copy()

        if (len(available_attributes) == 1):
            self.attribute = available_attributes[0]
            self.children = splits[0]
            del available_attributes_for_children[0]
        else:
            max_gain_ratio_index = 0
            max_gain_ratio = 0
            for index in range(len(available_attributes)):
                gain_ratio = self.gain_ratio(splits[index])
                if (gain_ratio > max_gain_ratio):
                    gain_ratio
                    max_gain_ratio = gain_ratio
                    max_gain_ratio_index = index

            self.attribute = available_attributes[max_gain_ratio_index]
            self.children = splits[max_gain_ratio_index]

            del available_attributes_for_children[max_gain_ratio_index]

        for child in self.children:
            child.split(available_attributes_for_children)

the `gain_ratio` method will be responsible for calculating the `gain_ratio` of the self node with possible children nodes: 

In [ ]:
    def gain_ratio(self, nodes: list) -> float:
        gain_info = self.entropy(self)
        for node in nodes:
            if (len(node.objects) == 0):
                continue

            entropy = self.entropy(node)
            ratio = len(node.objects) / len(self.objects)
            gain_info -= ratio*entropy

        split_info = 0
        for node in nodes:
            if (len(node.objects) == 0):
                continue

            ratio = len(node.objects) / len(self.objects)
            split_info -= ratio * log(ratio, 2)

        if (split_info == 0):
            return 0

        return gain_info / split_info

the `get_label` function is responsible for getting the class lable for one sample test object.

In [ ]:
    def get_label(self, obj: Object) -> int:
        if (self.label != -1):
            return self.label

        attribute_value = obj.attributes[self.attribute]
        return self.children[attribute_value].get_label(obj)

here `get_training_error_counts` is responsible for recursively calculates number of errors in training objects with calculated `label`.

In [ ]:
    def get_training_error_counts(self) -> int:
        if (self.label != -1):
            matches = filter(
                lambda obj: obj.attributes[-1] != self.label, self.objects)
            return len(list(matches))

        count = 0

        for child in self.children:
            count += child.get_training_error_counts()

        return count

the `majority` is a `static` method for calculating label of the leaf node by calculating majority label class of node objects.  

In [ ]:
    @staticmethod
    def majority(objects: list[Object]) -> int:
        counts = [0] * len(attributes[-1])

        for obj in objects:
            counts[obj.attributes[-1]] += 1

        max_value, max_index = -1, -1
        for index in range(len(counts)):
            if (counts[index] > max_value):
                max_value = counts[index]
                max_index = index

        return max_index

the `entropy` method is also a `static` one which is responsible for calculating entropy for a given node.

In [ ]:
    @staticmethod
    def entropy(node) -> float:
        entropy = 0.0

        for label in range(len(attributes[-1])):
            matches = filter(
                lambda obj: obj.attributes[-1] == label, node.objects)

            ratio = len(list(matches)) / len(node.objects)
            if (ratio != 0):
                entropy -= ratio * log(ratio, 2)

        return entropy

at the `main.py` as entrypoint of the application we are inducting the model.

here we first import our written files:

In [ ]:
from objects import Object
from attributes import attributes
from node import Node

then we are reading the contents of data and convert them into `Object` class:

In [ ]:
file = open("./static/car.data", "r")
lines = file.readlines()

objects = []

for line in lines:
    splits = line.removesuffix('\n').split(',')
    objects.append(Object(splits))

then we are using 90% of the objects as our training set and calculating the training_error and testing_error:

In [ ]:
max_depth = len(attributes) - 1

division_number = round(len(objects) * (90/100))
trainee_objects = objects[:division_number]
test_objects = objects[division_number:]

root = Node(trainee_objects, max_depth)
available_attributes = list(range(0, max_depth))
root.split(available_attributes)

training_error = root.get_training_error_counts()
print(f'training error: {(training_error/division_number)*100}')

testing_error = 0
for test_object in test_objects:
    label = root.get_label(test_object)
    if (test_object.attributes[-1] != label):
        testing_error += 1

print(f'test error: {(testing_error/(len(objects)-division_number))*100}')

then we use k-fold cross validation method as our induction strategy and calculate the average errors:

In [ ]:

def k_fold(k: int, max_depth: int = len(attributes)-1) -> float:
    k_fold_division_count = round(len(objects) / k)
    k_fold_training_error = 0
    k_fold_testing_error = 0

    for index in range(k):
        start = k_fold_division_count * index
        end = k_fold_division_count*(index+1)
        trainee_objects = objects[0:start] + objects[end:]
        test_objects = objects[start:end]

        root = Node(trainee_objects, max_depth)
        available_attributes = list(range(0, len(attributes)-1))
        root.split(available_attributes)

        k_fold_training_error += root.get_training_error_counts()

        for test_object in test_objects:
            label = root.get_label(test_object)
            if (test_object.attributes[-1] != label):
                k_fold_testing_error += 1

    return (k_fold_testing_error/(k_fold_division_count))/k


print()
k_values = [3, 5, 10, 15, 20]
for value in k_values:
    test_error = k_fold(value) * 100
    print(f'test error with {value}-fold cross-validation: {test_error}')

as question asked, we are using k-fold with the provided `k` value above and if we run the application we will see:

### running with normal strategy (90% of the data as training) we have:

> test error: 35.83815028901734

### running with k-fold strategy we have the following result:

> test error with 3-fold cross-validation: 31.770833333333332
>
> test error with 5-fold cross-validation: 24.91329479768786
>
> test error with 10-fold cross-validation: 23.179190751445084
>
> test error with 15-fold cross-validation: 14.318840579710146
>
> test error with 20-fold cross-validation: 10.290697674418606

as you can see as we increase the number of folds the value of `test error` will be decresed.

In [ ]:
optimal_depth_value = 0
optimal_depth = float("inf")

print()
for depth in range(len(attributes) - 1):
    k = 10
    test_error = k_fold(k, depth+1)
    complexity_error = 0.05 * (depth+1) / max_depth
    sum_error = (test_error + complexity_error) * 100

    if (sum_error < optimal_depth):
        optimal_depth = sum_error
        optimal_depth_value = depth+1

    print(
        f'test error with {k}-fold cross-validation and depth: {depth+1}: {sum_error}')

print()
print(
    f'maximum depth is: {max_depth}, optimal depth is: {optimal_depth_value}')


here we are calculating optimal depth by calculating the complexity of the model:

> test error with 10-fold cross-validation and depth: 1: 30.775529865125247

> test error with 10-fold cross-validation and depth: 2: 27.62042389210019

> test error with 10-fold cross-validation and depth: 3: 28.800578034682083

> test error with 10-fold cross-validation and depth: 4: 28.82466281310212

> test error with 10-fold cross-validation and depth: 5: 27.46146435452794

> test error with 10-fold cross-validation and depth: 6: 28.179190751445088

> maximum depth is: 6, optimal depth is: 5

as you can see, to avoid model overfitting it's better to calculate model complexity and determine the optimal depth as you can see here that optimal depth is not always the maximum one.